In [1]:
import json
import pandas
from datetime import datetime
from pymongo import MongoClient
from bson import json_util

#### Подключение к БД:

In [2]:
client = MongoClient("localhost:27017")
db = client['auto-programmer-db']
firmware_collection = db['firmware_projects_arch']
collection_list = db.list_collection_names()

if collection_list is not None:
    print("Подключение к MongoDB успешно")
else:
    print("Не удалось подключиться к MongoDB.")

Подключение к MongoDB успешно


#### Поиск документа по запросу:

In [3]:
def find_documents(query:dict):
    """
    Функция для поиска документов по заданному запросу.
    
    Parameters:
        query(dict): объект типа словарь, содержащий условие поиска
        
    Returns:
        json_documents: итерируемый объект JSON, содержащий документы, удовлетворяющие запросу
        
    Example_usage: 
        find_documents({"project_name":"td_ble_523"})
    """
    cursor = list(firmware_collection.find(query))
    if cursor:
        json_documents = json_util.dumps(cursor)
        return json.loads(json_documents)
        return json_documents
    else:
        return 'Документы не найдены'

Получим документ:

In [4]:
my_document = find_documents({"project_name":"td_ble_523"})

#### Получение списка уникальных имен:

In [5]:
def find_unique_project_names(collection, timestamp:str = None):
    """
    Функция для поиска уникальных имен проектов.
    
    Parameters:
        firmware_collection(pymongo.collection.Collection): Коллекция MongoDB, из которой нужно извлечь данные. 
        timestamp(str, optional): опциональный параметр, при наличии которого будет фильтрация только за указанную дату
                        (по умолчанию за все время)
        
    Returns:
       unique_project_names(list): возвращает список с уникальными именами проектов
        
    Example_usage: 
        find_unique_project_names(firmware_collection)
        find_unique_project_names(firmware_collection, '2024-03-21')
        
    """
    query = {}
    if timestamp:
        query['timestamp'] = {'$regex': timestamp}

    unique_project_names = firmware_collection.distinct('project_name', query)
    return unique_project_names

Получим список уникальных имен:

In [6]:
find_unique_project_names(firmware_collection, '2024-03-21')

['td_ble_523']

#### Функция для возврата словаря mac:serial в заданном промежутке времени:

In [35]:
def get_serial_mac_pairs(collection, start_date, end_date):
    """
    Функция для извлечения пар серийный номер - MAC-адрес за указанный промежуток времени.

    Parameters:
        collection (pymongo.collection.Collection): Коллекция MongoDB, из которой нужно извлечь данные. 
        start_date (str): Начальная дата в формате 'YYYY-MM-DD'.
        end_date (str): Конечная дата в формате 'YYYY-MM-DD'.
        
    Returns:
       serial_mac_pairs (dict): Cловарm, который содержит пары значений serial_number:mac_address.
       
    Example_usage:
        get_serial_mac_pairs(firmware_collection, "2024-03-20", "2024-03-24")

    """
    query = {'timestamp': {'$gte': start_date,
                           '$lte': end_date}}
    projection = {'_id': 0, 'plates': 1}

    cursor = collection.find(query, projection)

    serial_mac_dict = {}

    for document in cursor:
        plates = document.get('plates', {})
        for plate_info in plates.values():
            serial_number = plate_info.get('serial_number')
            mac_address = plate_info.get('mac_address')
            if serial_number and mac_address:
                serial_mac_dict[serial_number] = mac_address

    if serial_mac_dict:
        return serial_mac_dict
    else:
        return {'Документы за выбранные даты найдены'}

Получим словарь серийников и mac-адресов:

In [37]:
print(get_serial_mac_pairs(firmware_collection, "2024-03-22", "2024-03-24"))

{100153: 'A7:D6:8E:73:86:C5 C5:86:73:E', 100154: '04:44:3A:65:A0:DE DE:A0:65:A', 100155: 'A7:D6:8E:73:86:C5 C5:86:73:E', 100156: '1C:0C:49:B7:0E:C0 C0:0E:B7:9', 100157: 'A7:D6:8E:73:86:C5 C5:86:73:E', 100158: '1C:0C:49:B7:0E:C0 C0:0E:B7:9', 100159: 'F7:9E:69:40:2A:C5 C5:2A:40:9', 100160: '26:64:E1:B6:15:FF FF:15:B6:1', 100161: '06:80:D5:FD:CE:DA DA:CE:FD:5', 100162: 'A7:D6:8E:73:86:C5 C5:86:73:E', 100163: '//'}


def get_serial_mac_pairs(json_doc):
    serial_mac_pairs = []
    for document in json_doc:
        timestamp = document.get('timestamp')
        for plate_key, plate_value in document['plates'].items():
            if 'serial_number' in plate_value and 'mac_address' in plate_value:
                serial_number = plate_value.get('serial_number')
                mac_address = plate_value.get('mac_address')
                serial_mac_pairs.append({'serial_number': serial_number, 'mac_address': mac_address, 'timestamp': timestamp})
    return serial_mac_pairs

serial_mac_pairs = get_serial_mac_pairs(my_query)
serial_mac_pairs